In [173]:
import time, datetime
import re
import os
import pandas as pd
import numpy as np
import lightgbm as lgb
import matplotlib.pyplot as plt
from matplotlib import offsetbox
from pathlib import Path
from sklearn.preprocessing import Imputer, StandardScaler, MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPRegressor
from sklearn.externals import joblib
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split, GridSearchCV, KFold, LeaveOneOut
from sklearn.manifold import TSNE
from sklearn.cluster import DBSCAN, KMeans
from sklearn.metrics.pairwise import paired_distances
from sklearn.svm import SVR

In [174]:
# Load Data

train_file_path = '/home/data/1.生产参数记录表/生产参数记录表-2018年4月/'
test_file_path = '/home/data/1.生产参数记录表/生产参数记录表-2018年5月/'
temp_file_path = '/home/jovyan/work/'

X_list =   ['D101硫酸泵电流-Raw.csv',          'T101到T102洗涤液流量-Raw.csv',
            'E301干燥窑电流-Raw.csv',          'T101进口稀磷酸流量-Raw.csv',
            'E301干燥窑转速-Raw.csv',          'T102Һλ-Raw.csv',
            'E301干燥窑进口烟气温度-Raw.csv',  'T102进口浓磷酸流量-Raw.csv',
            'F101热风炉炉膛温度-Raw.csv',      'Z220造粒机电流-Raw.csv',
            'F101热风炉鼓风压力-Raw.csv',      'Z221进口液氨压力-Raw.csv',
            'F101鼓风机电流-Raw.csv',          'Z221进口液氨流量-Raw.csv',
            'G132洗涤塔液位-Raw.csv',          'Z230进口洗涤液压力-Raw.csv',
            'L101斗提机电流-Raw.csv',          'Z230进口洗涤液浓度-Raw.csv',
            'L102返料皮带电流-Raw.csv',        'Z230进口液氨压力-Raw.csv',
            'L103斗提机电流-Raw.csv',          'Z230进口液氨流量-Raw.csv',
            'S101A振网筛电流-Raw.csv',         'Z230进口硫酸压力-Raw.csv',
            'S101B振网筛电流-Raw.csv',         '内染剂泵流量-Raw.csv',
            'S102A破碎机电流1-Raw.csv',        '成品重量-Raw.csv',
            'S102A破碎机电流2-Raw.csv',        '浓硫酸流量-Raw.csv',
            'S102B破碎机电流1-Raw.csv',        '液氨压力-Raw.csv',
            'S102B破碎机电流2-Raw.csv',        '液氨流量-Raw.csv',
            'T101Һλ-Raw.csv',                 '返料重量-Raw.csv']

train_label_file = '/home/data/2.产品检验报告/产品检验报告2018-4-1.csv'
test_label_file = '/home/data/2.产品检验报告/产品检验报告2018-5-1-sample.csv'

train_df = {}
test_df = {}
for f in X_list:
    train_df[f] = pd.read_csv(train_file_path+f, header='infer', encoding='gbk')
    test_df[f] = pd.read_csv(test_file_path+f, header='infer', encoding='gbk')

y_train = pd.read_csv(train_label_file, header='infer', encoding='gbk')
y_test = pd.read_csv(test_label_file, header='infer', encoding='gbk')

model_path = '/home/model/'

In [ ]:
# Preprocessing 1: Add corresponding y_id to X dataframe

if not os.path.exists(temp_file_path+'train_with_id/'): 
    os.makedirs(temp_file_path+'train_with_id/')
if not os.path.exists(temp_file_path+'test_with_id/'): 
    os.makedirs(temp_file_path+'test_with_id/')
    
y_train = pd.read_csv(train_label_file, header='infer', encoding='gbk')
y_test = pd.read_csv(test_label_file, header='infer', encoding='gbk')
    
# ———————————————— 训练集id对齐 ————————————————
time_period = []
for t in y_train.iloc[:,1]:    # 训练集label所属时间段
    d = re.search(r'\d+.\d+', t).group(0)   # 匹配日期
    t1 = re.search(r'(\d{2}:\d{2}) - (\d{2}:\d{2})', t).group(1)   # 匹配起始时间点
    t2 = re.search(r'(\d{2}:\d{2}) - (\d{2}:\d{2})', t).group(2)   # 匹配结束时间点
    strtime = '2018.%s %s:00'%(d,t1)  
    y_ts1 = int(time.mktime(time.strptime(strtime, "%Y.%m.%d %H:%M:%S")))  # 转化为时间戳
    strtime = '2018.%s %s:00'%(d,t2)
    y_ts2 = int(time.mktime(time.strptime(strtime, "%Y.%m.%d %H:%M:%S")))
    if t2 == '00:00':          # 结束时间点为0点的，日期加1
        y_ts2 += 3600*24
    time_period.append((y_ts1, y_ts2))

for f in X_list:
    if os.path.exists(temp_file_path+'train_with_id/'+f):   # 文件已存在则跳过
        continue
    print('handling %s file of trainset...'%f)
    train_df[f]['id'] = None
    for i in range(len(train_df[f])):  
        if i % 1000 == 0:
            print('handling %d/%d record'%(i, len(train_df[f])))
        if type(train_df[f].iloc[i,1]) == str:    # 排除缺失值
            strtime = re.search(r'\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}',train_df[f].iloc[i,1]).group(0)  # 提取样本数据的测定时间点
            x_ts = int(time.mktime(time.strptime(strtime, "%Y-%m-%d %H:%M:%S")))                   # 转化为时间戳
            for j in range(len(time_period)):     # 遍历label数据所有的时间段，找出x对应的时间
                if time_period[j][0]<=x_ts and x_ts<=time_period[j][1]:
                    train_df[f].loc[i,'id'] = y_train.iloc[j,0]
                    break
    train_df[f].to_csv(temp_file_path+'train_with_id/'+f, index = False, encoding='utf-8')

# ———————————————— 测试集id对齐 ————————————————
time_period = []
for t in y_test.iloc[:,1]:    # 训练集label所属时间段
    d = re.search(r'\d+.\d+', t).group(0)   # 匹配日期
    t1 = re.search(r'(\d{2}:\d{2}) - (\d{2}:\d{2})', t).group(1)   # 匹配起始时间点
    t2 = re.search(r'(\d{2}:\d{2}) - (\d{2}:\d{2})', t).group(2)   # 匹配结束时间点
    strtime = '2018.%s %s:00'%(d,t1)  
    y_ts1 = int(time.mktime(time.strptime(strtime, "%Y.%m.%d %H:%M:%S")))  # 转化为时间戳
    strtime = '2018.%s %s:00'%(d,t2)
    y_ts2 = int(time.mktime(time.strptime(strtime, "%Y.%m.%d %H:%M:%S")))
    if t2 == '00:00':          # 结束时间点为0点的，日期加1
        y_ts2 += 3600*24
    time_period.append((y_ts1, y_ts2))

for f in X_list:
    if os.path.exists(temp_file_path+'test_with_id/'+f):   # 文件已存在则跳过
        continue
    print('handling %s file of testset...'%f)
    test_df[f]['id'] = None
    for i in range(len(test_df[f])):  
        if i % 1000 == 0:
            print('handling %d/%d record'%(i, len(test_df[f])))
        if type(test_df[f].iloc[i,1]) == str:    # 排除缺失值
            strtime = re.search(r'\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}',test_df[f].iloc[i,1]).group(0)  # 提取样本数据的测定时间点
            x_ts = int(time.mktime(time.strptime(strtime, "%Y-%m-%d %H:%M:%S")))                   # 转化为时间戳
            for j in range(len(time_period)):     # 遍历label数据所有的时间段，找出x对应的时间
                if time_period[j][0]<=x_ts and x_ts<=time_period[j][1]:
                    test_df[f].loc[i,'id'] = y_test.iloc[j,0]
                    break
    test_df[f].to_csv(temp_file_path+'test_with_id/'+f, index = False, encoding='utf-8')

    print('done.')

In [ ]:
# Preprocessing 2: 取36个检测属性的均值、方差等统计特征作为训练特征

if not os.path.exists(temp_file_path+'preprocessed_data/'): 
    os.makedirs(temp_file_path+'preprocessed_data/')
    
y_train = pd.read_csv(train_label_file, header='infer', encoding='gbk')
y_test = pd.read_csv(test_label_file, header='infer', encoding='gbk')
    
train_df_with_id = {}
test_df_with_id = {}
for f in X_list:
    train_df_with_id[f] = pd.read_csv(temp_file_path+'train_with_id/'+f,  header='infer', encoding='utf-8')
    test_df_with_id[f] = pd.read_csv(temp_file_path+'test_with_id/'+f,  header='infer', encoding='utf-8')

train_data_df = pd.DataFrame.copy(y_train)
test_data_df = pd.DataFrame.copy(y_test)
    
for f in X_list:
    for i in range(len(train_data_df)):
        df = train_df_with_id[f]
        target_data = df[df['id']==train_data_df.loc[i,'id']].iloc[:,2]
        mean = target_data.mean()
        std = target_data.std()
        target_data[target_data<mean-3*std] = mean        # 以均值替代异常值
        target_data[target_data>mean+3*std] = mean
        
        train_data_df.loc[i,f+'_mean'] = target_data.mean()   # 取均值
#         train_data_df.loc[i,f+'_std'] = target_data.std()     # 取方差
#         train_data_df.loc[i,f+'_max'] = target_data.max() # 取最大值
#         train_data_df.loc[i,f+'_min'] = target_data.min() # 取最小值
#         train_data_df.loc[i,f+'_median'] = target_data.median() # 中位数
#         train_data_df.loc[i,f+'_max-min'] = train_data_df.loc[i,f+'_max']-train_data_df.loc[i,f+'_min'] 
                                    
        if i < len(test_data_df):
            df = test_df_with_id[f]
            target_data = df[df['id']==test_data_df.loc[i,'id']].iloc[:,2]
            mean = target_data.mean()
            std = target_data.std()
            target_data[target_data<mean-3*std] = mean        # 以均值替代异常值
            target_data[target_data>mean+3*std] = mean
            
            test_data_df.loc[i,f+'_mean'] = target_data.mean()   # 取均值
#             test_data_df.loc[i,f+'_std'] = target_data.std()     # 取方差
#             test_data_df.loc[i,f+'_max'] = target_data.max()    # 取最大值
#             test_data_df.loc[i,f+'_min'] = target_data.min()    # 取最小值
#             test_data_df.loc[i,f+'_median'] = target_data.median() # 中位数
#             test_data_df.loc[i,f+'_max-min'] = test_data_df.loc[i,f+'_max']-test_data_df.loc[i,f+'_min']
    print('finish '+ f)
    
train_data_df.to_csv(temp_file_path+'preprocessed_data/train_mean.csv', index = False, encoding='utf-8')
test_data_df.to_csv(temp_file_path+'preprocessed_data/test_mean.csv', index = False, encoding='utf-8')
print('done')

In [177]:
# Train 1: 准备训练数据，请先运行预处理模块产生所需数据

train_data_df = pd.read_csv(temp_file_path+'preprocessed_data/train_mean.csv', header='infer', encoding='utf-8')
test_data_df = pd.read_csv(temp_file_path+'preprocessed_data/test_mean.csv', header='infer', encoding='utf-8')

# K = None         # 保留特征个数，None表示全部保留
# r_stds = []     # 保存各特征的归一化标准差，筛选出归一化标准差最大的前K各特征
# for i in range(7, len(train_data_df.columns)):
#     mean = train_data_df.iloc[:,i].mean()
#     r_std = (train_data_df.iloc[:,i]/mean).std()    # 计算归一化之后的标准差
#     r_stds.append((r_std,i))
# r_stds.sort(reverse = True)
# f_index = list(list(zip(*r_stds[:K]))[1])  # 获得前K重要的特征列index

new_train_data_df = pd.merge(train_data_df.iloc[:,:7], train_data_df.iloc[:, f_index], left_index=True, right_index=True)
new_test_data_df = pd.merge(test_data_df.iloc[:,:7], test_data_df.iloc[:, f_index], left_index=True, right_index=True)

# new_train_data_df['返料比'] = new_train_data_df['返料重量-Raw.csv_mean']/new_train_data_df['成品重量-Raw.csv_mean']
# new_test_data_df['返料比'] = new_test_data_df['返料重量-Raw.csv_mean']/new_test_data_df['成品重量-Raw.csv_mean']
# new_train_data_df['返料差'] = new_train_data_df['返料重量-Raw.csv_mean']-new_train_data_df['成品重量-Raw.csv_mean']
# new_test_data_df['返料差'] = new_test_data_df['返料重量-Raw.csv_mean']-new_test_data_df['成品重量-Raw.csv_mean']

X_train = new_train_data_df.iloc[:,7:].values
X_test = new_test_data_df.iloc[:,7:].values

valid_sample_id = [i for i in range(len(X_train))]    # 移除特征值全空的样本
for i in range(len(X_train)):
    if np.isnan(X_train[i]).all():
        valid_sample_id.remove(i)
X_train = X_train[valid_sample_id]

imp = Imputer(missing_values='NaN', strategy='mean', axis=0)  # 以均值填补缺失值
X_train = imp.fit_transform(X_train)
X_test = imp.fit_transform(X_test)

y_train_df = pd.read_csv(train_label_file, header='infer', encoding='gbk')
y_test_df = pd.read_csv(test_label_file, header='infer', encoding='gbk')
y_train = y_train_df.iloc[:,2:7].values[valid_sample_id].T
# y_train = y_train_df.iloc[:,2:7].values.T
y_test = y_test_df.iloc[:,2:7].values.T

# 通过对训练集聚类来对分层样本
random_seed = 0
cluster_model = KMeans(n_clusters=6, random_state=random_seed).fit(X_train)  # KMeans聚类
cluster_labels = cluster_model.labels_ 

print(X_train.shape, y_train.shape)

(141, 36) (5, 141)


/opt/conda/lib/python3.6/site-packages/sklearn/utils/deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


In [ ]:
# 可视化聚类效果
def plot_embedding(X, title=None): 
    x_min, x_max = np.min(X, 0), np.max(X, 0) 
    X = (X - x_min) / (x_max - x_min)    #正则化 
    # print(X)
    plt.figure(figsize=(10, 10)) 
    ax = plt.subplot(111) 
    for i in range(X.shape[0]): 
        plt.text(X[i, 0], X[i, 1], '*', color=plt.cm.Set1(labels[i]), fontdict={'weight': 'bold', 'size': 12}) 
    plt.xticks([]), plt.yticks([]) 
    if title is not None: 
        plt.title(title)

cluster_model = KMeans(n_clusters=6, random_state=0).fit(X_train)
cluster_labels = cluster_model.labels_
print(calinski_harabaz_score(X_train, cluster_labels))

tsne = TSNE(n_components=2, init='pca', random_state=0)
X_train_tsne = tsne.fit_transform(X_train)
plot_embedding(X_train_tsne)
plt.show()

In [241]:
# Train 2-2*: lightGBM 留一法验证模型
label_list = y_test_df.columns.values[2:].tolist()
feature_list = train_data_df.columns[7:]

epochs = [30, 125, 30, 70, 25]
for i in range(len(label_list)):
    k = 0
    rmse = []
    loo = LeaveOneOut()    # 留一法划分数据集
    for train_index, valid_index in loo.split(X_train):
        k += 1
        X_train_t, X_valid_t = X_train[train_index], X_train[valid_index]
        y_train_t, y_valid_t = y_train.T[train_index], y_train.T[valid_index]
        
        model = lgb.LGBMRegressor(objective='regression',max_depth=-1, num_leaves=64,learning_rate=0.05,\
                                  n_estimators=epochs[i], bagging_freq=5, bagging_feaction=1.0, feature_fraction=0.7)
        model.fit(X_train_t, y_train_t.T[i], eval_metric='rmse')
        y_pred_valid = model.predict(X_valid_t)
        rmse.append(mean_squared_error(y_valid_t.T[i], y_pred_valid) ** 0.5)               # 计算在验证集上的RMSE
#         if k%10 == 0:    
#             print('training %s model %d times...'%(label_list[i], k))

    score = np.mean(rmse)
    print('The RMSE on validation set of %s: %.5f'%(label_list[i],score)) 

The RMSE on validation set of phosphorus_content: 0.26570
The RMSE on validation set of nitrogen_content: 0.13536
The RMSE on validation set of total_nutrient: 0.25411
The RMSE on validation set of water_content: 0.22303
The RMSE on validation set of particle_size: 2.49567


In [242]:
# Train 2-2*: lightGBM 全数据集训练模型，直接产生测试结果

Re_train_model = True     # 是否从头训练模型，False的情况下，会尝试直接加载已存在的模型

if not os.path.exists(temp_file_path+'model/'):
    os.makedirs(temp_file_path+'model/')

print(X_train.shape, y_train.shape)

label_list = y_test_df.columns.values[2:].tolist()

result = {}
epochs = [30, 125, 30, 70, 25]
for i in range(len(label_list)):
    if not Re_train_model and Path('%slightGBM_%s.model'%(temp_file_path+'model/', label_list[i])).exists():
        print('Start loading %s model...'%label_list[i])
        model = joblib.load('%slightGBM_%s.model'%(temp_file_path+'model/', label_list[i]))
    else:
        print('Start training %s model...'%label_list[i])
        model = lgb.LGBMRegressor(objective='regression',max_depth=-1, num_leaves=64,learning_rate=0.05,\
                                  n_estimators=epochs[i], bagging_freq=5, bagging_feaction=1.0, feature_fraction=0.7)
        model.fit(X_train, y_train[i])     # 全数据集训练模型
        print('Start saving %s model...'%label_list[i])
        joblib.dump(model, '%slightGBM_%s.model'%(temp_file_path+'model/', label_list[i]))
        
    print('Start predicting %s model...'%label_list[i])
    result[label_list[i]] = model.predict(X_test)
    
    # ———————————————————————— 输出特征重要性 —————————————————————
#     print('【Feature importances】')             
#     importance = sorted(list(zip(list(model.feature_importances_), feature_list)), reverse=True) # 降序排序
#     for f_id in range(len(feature_list)):
#         if importance[f_id][0] == 0:
#             break
#         print(importance[f_id][1],": ",importance[f_id][0])
#     print('others: 0\n')
    # —————————————————————————————————————————————————————
    
result_df = pd.DataFrame.copy(y_test_df)
for col in result.keys():
    result_df[col] = result[col]

result_df = result_df.drop(['product_batch'],axis=1)
result_df.to_csv(temp_file_path+'LGBM_result.csv', index=False, encoding='utf-8')
print('done!')

(141, 36) (5, 141)
Start training phosphorus_content model...
Start saving phosphorus_content model...
Start predicting phosphorus_content model...
Start training nitrogen_content model...
Start saving nitrogen_content model...
Start predicting nitrogen_content model...
Start training total_nutrient model...
Start saving total_nutrient model...
Start predicting total_nutrient model...
Start training water_content model...
Start saving water_content model...
Start predicting water_content model...
Start training particle_size model...
Start saving particle_size model...
Start predicting particle_size model...
done!


In [243]:
# Train 2-4*: SVR 留一法验证模型
label_list = y_test_df.columns.values[2:].tolist()
feature_list = train_data_df.columns[7:]

C_list = [1, 1, 1, 1, 1]
for i in range(len(label_list)):
    k = 0
    rmse = []
    loo = LeaveOneOut()    # 留一法划分数据集
    for train_index, valid_index in loo.split(X_train):
        k += 1
        X_train_t, X_valid_t = X_train[train_index], X_train[valid_index]
        y_train_t, y_valid_t = y_train.T[train_index], y_train.T[valid_index]
        
        model = SVR(gamma='scale', C=C_list[i])
        model.fit(X_train_t, y_train_t.T[i])
        y_pred_valid = model.predict(X_valid_t)
        rmse.append(mean_squared_error(y_valid_t.T[i], y_pred_valid) ** 0.5)               # 计算在验证集上的RMSE
#         if k%10 == 0:    
#             print('training %s model %d times...'%(label_list[i], k))

    score = np.mean(rmse)
    print('The RMSE on validation set of %s: %.5f'%(label_list[i],score)) 

The RMSE on validation set of phosphorus_content: 0.26813
The RMSE on validation set of nitrogen_content: 0.14766
The RMSE on validation set of total_nutrient: 0.27125
The RMSE on validation set of water_content: 0.22623
The RMSE on validation set of particle_size: 2.38478


In [244]:
# Train 2-4*: SVR 全数据集训练模型，直接产生测试结果

Re_train_model = True     # 是否从头训练模型，False的情况下，会尝试直接加载已存在的模型

if not os.path.exists(temp_file_path+'model/'):
    os.makedirs(temp_file_path+'model/')
    
label_list = y_test_df.columns.values[2:].tolist()

result = {}
C_list = [1, 1, 1, 1, 1]
print(X_train.shape, y_train.shape)
for i in range(len(label_list)):                    # 为每个指标单独训练一个模型
    if not Re_train_model and Path('%sSVR_%s.model'%(temp_file_path+'model/', label_list[i])).exists():
        print('Start loading %s model...'%label_list[i])
        model = joblib.load('%sSVR_%s.model'%(temp_file_path+'model/', label_list[i]))
    else:
        model = SVR(gamma='scale', C=C_list[i])
        model.fit(X_train, y_train[i])
        joblib.dump(model, '%sSVR_%s.model'%(temp_file_path+'model/', label_list[i]))   # 保存模型
        result[label_list[i]] = model.predict(X_test)

result_df = pd.DataFrame.copy(y_test_df)    
for col in result.keys():
    result_df[col] = result[col]

result_df = result_df.drop(['product_batch'],axis=1)
result_df.to_csv(temp_file_path+'SVR_result.csv', index=False, encoding='utf-8')
print('done!')

(141, 36) (5, 141)
done!


In [245]:
# Train 2-5*: KNN距离加权 留一法验证模型
label_list = y_test_df.columns.values[2:].tolist()
feature_list = train_data_df.columns[7:]

def KNN_weighted_regression(X, y, test_sample, k = 3, metric = 'euclidean'):
    KNN_index = [i for i in range(len(X))]
    KNN_distance = paired_distances(X, np.asarray([test_sample]*len(X)).reshape((len(X),len(test_sample))), metric=metric)
    order = sorted(zip(KNN_distance, KNN_index))
    topK_distance, topK_index = zip(*order)
    if topK_distance[0] == 0:
        topK_distance = topK_distance[1:]
        topK_index = topK_index[1:]
    topK_distance = topK_distance[:k]
    topK_index = topK_index[:k]
    y_pred = sum([y[topK_index[i]]*(1/topK_distance[i]) for i in range(len(topK_index))])
    y_pred /= sum(1/np.asarray(topK_distance))
    return y_pred

for i in range(len(label_list)):
    k = 0
    rmse = []
    loo = LeaveOneOut()    # 留一法划分数据集
    for train_index, valid_index in loo.split(X_train):
        k += 1
        X_train_t, X_valid_t = X_train[train_index], X_train[valid_index]
        y_train_t, y_valid_t = y_train.T[train_index], y_train.T[valid_index]
        
        y_pred_valid = KNN_weighted_regression(X_train_t, y_train_t.T[i] ,X_valid_t[0], k=len(X_train_t), metric='cosine')
        rmse.append(mean_squared_error(y_valid_t.T[i], [y_pred_valid]) ** 0.5)               # 计算在验证集上的RMSE
#         if k%10 == 0:    
#             print('training %s model %d times...'%(label_list[i], k))

    score = np.mean(rmse)
    print('The RMSE on validation set of %s: %.5f'%(label_list[i],score)) 

The RMSE on validation set of phosphorus_content: 0.27286
The RMSE on validation set of nitrogen_content: 0.16811
The RMSE on validation set of total_nutrient: 0.26776
The RMSE on validation set of water_content: 0.22004
The RMSE on validation set of particle_size: 2.48000


In [246]:
# Train 2-5*: KNN距离加权 全数据集训练模型，直接产生测试结果

label_list = y_test_df.columns.values[2:].tolist()

result = {}
print(X_train.shape, y_train.shape)
for i in range(len(label_list)):                    # 为每个指标单独训练一个模型
    y_pred = [KNN_weighted_regression(X_train, y_train[i] , x, k=len(X_train), metric='cosine') for x in X_test]
    result[label_list[i]] = y_pred

result_df = pd.DataFrame.copy(y_test_df)    
for col in result.keys():
    result_df[col] = result[col]

result_df = result_df.drop(['product_batch'],axis=1)
result_df.to_csv(temp_file_path+'KNN_result.csv', index=False, encoding='utf-8')
print('done!')

(141, 36) (5, 141)
done!


In [252]:
# simple stacking 留一法找最优迭代

stacking_model = ['lightGBM','SVR', 'KNN']

label_list = y_test_df.columns.values[2:].tolist()

train_set_predict = []
test_set_predict = []

for m in stacking_model:
    if m == 'KNN':
        continue
    print('Start predict %s model...'%m)
    for i in range(len(label_list)):
        model = joblib.load('%s%s_%s.model'%(temp_file_path+'model/', m, label_list[i]))
        train_set_predict.append(model.predict(X_train))
        test_set_predict.append(model.predict(X_test))
if 'KNN' in stacking_model:
    print('Start predict %s model...'%'KNN')
    for i in range(len(label_list)):
        y_pred1 = [KNN_weighted_regression(X_train, y_train[i] , x, k=len(X_train), metric='cosine') for x in X_train]
        y_pred2 = [KNN_weighted_regression(X_train, y_train[i] , x, k=len(X_train), metric='cosine') for x in X_test]
        train_set_predict.append(y_pred1)
        test_set_predict.append(y_pred2)    
        
train_set_predict = np.asarray(train_set_predict)
test_set_predict = np.asarray(test_set_predict)

result = {}
rmse = []
model_type = 'LGBM'

epochs = [70, 100, 70, 70, 50]
print('Start train %s model...'%'stacking')
for i in range(len(label_list)):
    k = 0
    rmse = []
    loo = LeaveOneOut()    # 留一法划分数据集
    for train_index, valid_index in loo.split(X_train):
        k += 1
        X_train_s, X_valid_s = train_set_predict.T[train_index], train_set_predict.T[valid_index]
        y_train_s, y_valid_s = y_train.T[train_index], y_train.T[valid_index]
        if model_type == 'MLP': 
        # —————————————————--------—— MLP ——————————————--—————————————
            num_iter = [500, 500, 500, 500, 100]
            model = MLPRegressor(hidden_layer_sizes=(20), max_iter=num_iter[i], activation='relu', solver='lbfgs',
                                 random_state=0)
            model.fit(X_train_s, y_train_s.T[i])
        # —————————————————————————————————————————————————————-
        elif model_type == 'SVR':
        # —————————————————--------—— SVR ——————————————--—————————————
            model = SVR(gamma='scale')
            model.fit(X_train_s, y_train_s.T[i])
        # ————————————————————————————————————-—————————————————
        elif model_type == 'LR':
        # —————————————————--------——- LR ——————————————--—————————————
            model = LinearRegression(fit_intercept=True,normalize=False,copy_X=True,n_jobs=1)
            model.fit(X_train_s, y_train_s.T[i])
        # —————————————————--------————-——————————————--—————————————
        elif model_type == 'LGBM':
        # —————————————————--------—— LGBM ————————————————————————————
            model = lgb.LGBMRegressor(objective='regression',max_depth=-1, num_leaves=64,learning_rate=0.05, n_estimators=epochs[i])
            model.fit(X_train_s, y_train_s.T[i])

        # —————————————————--------————-——————————————--—————————————

        # —————————————————--------—— ANN ————————————————————————————
    #     model.add(Dense(10, input_shape=(X_train_s.shape[1],)))
    #     model.add(BatchNormalization())
    #     model.add(Activation('sigmoid'))
    #     model.add(Dense(5, input_shape=(X_train_s.shape[1],)))
    #     model.add(BatchNormalization())
    #     model.add(Activation('sigmoid'))
    #     model.add(Dense(1, activation='linear'))
    #     model.compile(optimizer='sgd', loss='mse')
    #     early_stopping = EarlyStopping(monitor='val_loss', patience=10)
    #     reduce_LR = ReduceLROnPlateau(monitor='val_loss', patience=5)
    #     checkPoint = ModelCheckpoint(filepath='%sstacking_%s.model'%(temp_file_path+'model/', label_list[i]),\
    #                                  monitor='val_loss', save_best_only=True)
    #     call_back_func = [early_stopping,reduce_LR, checkPoint]
    #     model.fit(X_train_s, y_train_s, validation_data=(X_valid_s, y_valid_s), epochs=30, callbacks=call_back_func)

    #     model = load_model(filepath='%sstacking_%s.model'%(temp_file_path+'model/', label_list[i]))
        # ————————————————————————————————————-——————————————————
   
        y_pred_valid = model.predict(X_valid_s)
        rmse.append(mean_squared_error(y_valid_s.T[i], y_pred_valid) ** 0.5)     # 计算在验证集上的RMSE
#         if k%10 == 0:    
#             print('training %s model %d times...'%(label_list[i], k))
     
    score = np.mean(rmse)
    print('The RMSE on validation set of %s: %.5f'%(label_list[i],score)) 
print('done!')    

Start predict lightGBM model...
Start predict SVR model...
Start predict KNN model...
Start train stacking model...
The RMSE on validation set of phosphorus_content: 0.16070
The RMSE on validation set of nitrogen_content: 0.05993
The RMSE on validation set of total_nutrient: 0.15313
The RMSE on validation set of water_content: 0.10503
The RMSE on validation set of particle_size: 1.71899
done!


In [253]:
# simple stacking 输出测试结果

stacking_model = ['lightGBM','SVR', 'KNN']

label_list = y_test_df.columns.values[2:].tolist()

train_set_predict = []
test_set_predict = []

for m in stacking_model:
    if m == 'KNN':
        continue
    print('Start predict %s model...'%m)
    for i in range(len(label_list)):
        model = joblib.load('%s%s_%s.model'%(temp_file_path+'model/', m, label_list[i]))
        train_set_predict.append(model.predict(X_train))
        test_set_predict.append(model.predict(X_test))
if 'KNN' in stacking_model:
    print('Start predict %s model...'%'KNN')
    for i in range(len(label_list)):
        y_pred1 = [KNN_weighted_regression(X_train, y_train[i] , x, k=len(X_train), metric='cosine') for x in X_train]
        y_pred2 = [KNN_weighted_regression(X_train, y_train[i] , x, k=len(X_train), metric='cosine') for x in X_test]
        train_set_predict.append(y_pred1)
        test_set_predict.append(y_pred2)            
            
        
train_set_predict = np.asarray(train_set_predict)
test_set_predict = np.asarray(test_set_predict)

result = {}
rmse = []
model_type = 'LGBM'
validation = False

X_train_s = train_set_predict.T
y_train_s = y_train.T
    
print(X_train_s.shape)
epochs = [70, 100, 70, 70, 50]
for i in range(len(label_list)):
    if model_type == 'MLP': 
    # —————————————————--------—— MLP ——————————————--—————————————
        num_iter = [500, 500, 500, 500, 100]
        model = MLPRegressor(hidden_layer_sizes=(20), max_iter=num_iter[i], activation='relu', solver='lbfgs',
                             random_state=0)
        model.fit(X_train_s, y_train_s.T[i])
    # —————————————————————————————————————————————————————-
    elif model_type == 'SVR':
    # —————————————————--------—— SVR ——————————————--—————————————
        model = SVR(gamma='scale')
        model.fit(X_train_s, y_train_s.T[i])
    # ————————————————————————————————————-—————————————————
    elif model_type == 'LR':
    # —————————————————--------——- LR ——————————————--—————————————
        model = LinearRegression(fit_intercept=True,normalize=False,copy_X=True,n_jobs=1)
        model.fit(X_train_s, y_train_s.T[i])
    # —————————————————--------————-——————————————--—————————————
    elif model_type == 'LGBM':
    # —————————————————--------—— LGBM ————————————————————————————
        model = lgb.LGBMRegressor(objective='regression',max_depth=-1, num_leaves=64,learning_rate=0.05, n_estimators=epochs[i])
        if validation:
            model.fit(X_train_s, y_train_s.T[i], eval_set=[(X_valid_s, y_valid_s.T[i])],eval_metric='rmse',\
                      early_stopping_rounds=10, verbose=False)
        else:
            model.fit(X_train_s, y_train_s.T[i], verbose=False)
    # —————————————————--------————-——————————————--—————————————
    result[label_list[i]] = model.predict(test_set_predict.T)

result_df = pd.DataFrame.copy(y_test_df)    
for col in result.keys():
    result_df[col] = result[col]

result_df = result_df.drop(['product_batch'],axis=1)
result_df.to_csv(temp_file_path+'stacking_result.csv', index=False, encoding='utf-8')
print('done!')

Start predict lightGBM model...
Start predict SVR model...
Start predict KNN model...
(141, 15)
done!


In [261]:
# result merge
df1 = pd.read_csv('0.288429.csv', header=0, encoding='gbk')
df2 = pd.read_csv('0.28992.csv', header=0, encoding='gbk')

df1['total_nutrient'] = df1['phosphorus_content'] + df1['nitrogen_content']
df2['total_nutrient'] = df2['phosphorus_content'] + df2['nitrogen_content']

weights = [0.288429, 0.28992]
inverse_sum = sum([1/w for w in weights])
for c in df1.columns.values[1:]:
    df1[c] = (df1[c] / weights[0] + df2[c] /weights[1])/inverse_sum

df1.to_csv('merge_result.csv', index=False, encoding='utf-8')